In [27]:
from datasets import list_datasets, load_dataset
from pprint import pprint

from huggingface_hub import notebook_login
notebook_login()

In [2]:
dataset = load_dataset('sem_eval_2018_task_1', 'subtask5.spanish')

Found cached dataset sem_eval_2018_task_1 (/home/juanjo/.cache/huggingface/datasets/sem_eval_2018_task_1/subtask5.spanish/1.1.0/a7c0de8b805f1988b118882fb289ccfbbeb9085c7820b6f046b5887e234af182)


  0%|          | 0/3 [00:00<?, ?it/s]

In [28]:
dataset2 = load_dataset('fmplaza/EmoEvent', use_auth_token=True)

No config specified, defaulting to: emo_event/es
Found cached dataset emo_event (/home/juanjo/.cache/huggingface/datasets/fmplaza___emo_event/es/1.1.0/05d5c93b8773e3d90b52effb7132b04cd8b2ccea8d4a0750989ec3aaeb59cad4)


  0%|          | 0/3 [00:00<?, ?it/s]

In [43]:
pd.DataFrame(dataset2['train'])

,id,event,tweet,offensive,label
0,a0c1a858-a9b8-4cb1-8a81-1602736ff5b8,GameOfThrones,La Gran Guerra de #JuegoDeTronos nos ha dejado...,NO,6
1,9b272817-a231-4f68-bdf4-3350d4919330,Venezuela,El golpe de Estado en #Venezuela está más lejo...,NO,6
2,4bd5b1e5-4b74-440a-82f4-c2567a241011,NotreDame,No tengo una foto en la catedral de #NotreDame...,NO,2
3,0bb9d7c9-d781-4684-890e-a94bfb50acc0,NotreDame,#NotreDame nunca llegue a visitar tan grandios...,NO,2
4,88749098-d539-4500-9209-0bbfae2b109c,GameOfThrones,A tomar por culo mi crush 😭😭😭😭😭#JuegoDeTronos,OFF,2
...,...,...,...,...,...
5718,c724c23b-2f6e-46f4-b740-0a7186349f60,Venezuela,"#Venezuela Querida, un dictador como Maduro so...",NO,6
5719,ee10fa89-f6ad-4f61-9bf7-d90604ce5b6b,NotreDame,#Notredame Impactantes imágenes que ha dejado ...,NO,2
5720,c0524970-cd8f-41a6-b9fa-ed052b23c297,Venezuela,¡ESCUCHA MADURO! Militares de #USA estarán ate...,NO,6
5721,f7b79584-d865-4e86-a83a-159792e98158,NotreDame,#NotreDame : 💔 💔 💔 Todo es efímero como el arc...,NO,6


In [1]:
import tweepy
import pandas as pd
import os
import unidecode
#import datetime
from datetime import timedelta, datetime 
from dateutil.relativedelta import relativedelta
import time

In [2]:
client = tweepy.Client(bearer_token = "AAAAAAAAAAAAAAAAAAAAAKKEcwEAAAAA1VgzT5upnu%2B5POQ3dbdUx0%2B5vfA%3DdslAW6ut31gZhQD9SjNKJ4scCFd8bHgbevp8eGcJprbd2LTrGf", wait_on_rate_limit = True)

In [3]:
df = pd.read_csv('../Data/corpus_marcado6emociones_extracted.csv')  

In [14]:
df[df['emocion'] == 'Ira']

,id,text,emocion
4098,464462822957912065,Ese momento que vas a lavarte los dientes y se...,Ira
4099,464726732491464704,Necesito el teléfono del creador del nivel 86 ...,Ira
4100,464731936796463104,A esta hora todos los que esperan el subte con...,Ira
4101,464757652061683712,"El Régimen Colectivos, GNB, PNB exacerbó al 60...",Ira
4102,464826450592673792,La mayor traición jamás esperada! Mi mejor ami...,Ira
...,...,...,...
5925,482101912741748738,#NadaMasFeoQue traducir las cosas a lo pocho.....,Ira
5926,482234398842961920,#nadamasfeoque posponer esas conversaciones de...,Ira
5927,464880366730235904,No entiendo cómo hay gente que le gusta econom...,Ira
5928,467069094899228672,@JessAhelle @espedevesa cómo no van a estar la...,Ira


In [26]:
df['text'][8821]

'le va a sacar foto al dibujo, me lo va a pasar, pero de todas formas va a venir a mi casa a darmelo;-; lo peor es que sabe donde vivo #miedo'

In [2]:
sel = pd.read_csv('../Data/SEL/SEL_full.txt', delimiter='\t', encoding= 'ISO-8859-1')

In [3]:
emociones = list(sel['Categoría'].unique())
emociones_dict ={}
for emocion in emociones:
    emociones_dict[emocion] = list(sel[sel['Categoría'] == emocion].sort_values(' PFA',  ascending=False).iloc[0:10]['Palabra'])
    for palabra in emociones_dict[emocion]:
        if not palabra.isascii():
            emociones_dict[emocion].append(unidecode.unidecode(palabra))

In [5]:
def get_tweets(emotion, words_list, number_tweets, start, end):
    emotion_words = '(' +' OR '.join(words_list) + ')'
    searched_tweets = [tweet for tweet in tweepy.Paginator(client.search_all_tweets, query = emotion_words + ' place_country:CO' ,start_time = start, end_time = end, 
                                max_results=500).flatten(limit=number_tweets)]
    tweets_df = pd.DataFrame(searched_tweets, columns = ['text'])
    tweets_df['emotion']= emotion
    tweets_df['date']= start
    return tweets_df

In [9]:
month = '04'
start_date = datetime.strptime('2021-' + month, '%Y-%m')
end_date = start_date + relativedelta(months=1)
df_list = []
while start_date < end_date:
    print(start_date)
    next_date = start_date + relativedelta(days=1)
    start_date_standard = start_date.isoformat("T") + "Z"
    next_date_standard = next_date.isoformat("T") + "Z"
    for i in emociones_dict.keys():
        print(i)
        df_list.append(get_tweets(i, emociones_dict[i], 100, start_date_standard, next_date_standard ))
        time.sleep(1)
    start_date = next_date
emociones_df = pd.concat(df_list)

2021-04-01 00:00:00
Alegría
Enojo
Miedo
Repulsión
Sorpresa
Tristeza
2021-04-02 00:00:00
Alegría
Enojo
Miedo
Repulsión
Sorpresa
Tristeza
2021-04-03 00:00:00
Alegría
Enojo
Miedo
Repulsión
Sorpresa
Tristeza
2021-04-04 00:00:00
Alegría
Enojo
Miedo
Repulsión
Sorpresa
Tristeza
2021-04-05 00:00:00
Alegría
Enojo
Miedo
Repulsión
Sorpresa
Tristeza
2021-04-06 00:00:00
Alegría
Enojo
Miedo
Repulsión
Sorpresa
Tristeza
2021-04-07 00:00:00
Alegría
Enojo
Miedo
Repulsión
Sorpresa
Tristeza
2021-04-08 00:00:00
Alegría
Enojo
Miedo
Repulsión
Sorpresa
Tristeza
2021-04-09 00:00:00
Alegría
Enojo
Miedo
Repulsión
Sorpresa
Tristeza
2021-04-10 00:00:00
Alegría
Enojo
Miedo
Repulsión
Sorpresa
Tristeza
2021-04-11 00:00:00
Alegría
Enojo
Miedo
Repulsión
Sorpresa
Tristeza
2021-04-12 00:00:00
Alegría
Enojo
Miedo
Repulsión
Sorpresa
Tristeza
2021-04-13 00:00:00
Alegría
Enojo
Miedo
Repulsión
Sorpresa
Tristeza
2021-04-14 00:00:00
Alegría
Enojo
Miedo
Repulsión
Sorpresa
Tristeza
2021-04-15 00:00:00
Alegría
Enojo
Miedo
Repulsió

In [11]:
emociones_df.to_csv('../Data/emociones_df.txt', index=False)

In [20]:
emociones_df = pd.read_csv('../Data/emociones_df.txt')

In [6]:
from transformers import BertForMaskedLM, BertTokenizer, BertForSequenceClassification

In [21]:

tokenizer = BertTokenizer.from_pretrained("/home/juanjo/BETO/pytorch/", do_lower_case=False)
model = BertForMaskedLM.from_pretrained("/home/juanjo/BETO/pytorch")
e = model.eval()

Some weights of the model checkpoint at /home/juanjo/BETO/pytorch were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
text = "[CLS] Para solucionar los [MASK] de Chile, el presidente debe [MASK] de inmediato. [SEP]"
masked_indxs = (4,11)

tokens = tokenizer.tokenize(text)
indexed_tokens = tokenizer.convert_tokens_to_ids(tokens)
tokens_tensor = torch.tensor([indexed_tokens])

predictions = model(tokens_tensor)[0]

for i,midx in enumerate(masked_indxs):
    idxs = torch.argsort(predictions[0,midx], descending=True)
    predicted_token = tokenizer.convert_ids_to_tokens(idxs[:5])
    print('MASK',i,':',predicted_token)

MASK 0 : ['problemas', 'conflictos', 'asuntos', 'males', 'temas']
MASK 1 : ['renunciar', 'actuar', 'intervenir', 'regresar', 'asumir']
